In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#optimization: AdamW
#model: microsoft/CodeBERT
#learning rate: 2e-5
#epoch: 7
#train_test_split=0.1, seed = 100
#batch_size = 16
#new_train_sample_5_size = 45088

In [3]:
!pip install torch
!pip install transformers
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import re
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer, RobertaConfig, RobertaModel
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss

In [5]:
#Load Train dataset
train = pd.read_csv('new_train_sample_5.csv', encoding = 'utf-8')
df = pd.DataFrame(train)

In [6]:
#Define Model
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
#tokenizer.truncation_side = 'left'

batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be 

In [7]:
def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)
  
def tokenized(examples):
  return tokenizer(examples['code1'],examples['code2'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [8]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['Unnamed: 0','code1','code2'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='similar',new_column_name='labels')
encoded_dataset
print(encoded_dataset[1])
print(len(encoded_dataset))
print(tokenizer.tokenize(dataset['code2'][1]))

Using custom data configuration default-1f3dbaafea062f16


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1f3dbaafea062f16/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (824 > 512). Running this sequence through the model will result in indexing errors


{'labels': 0, 'input_ids': [0, 487, 5457, 6979, 1640, 46797, 49338, 50118, 1990, 939, 11, 1186, 1640, 134, 6, 158, 3256, 50118, 50117, 1990, 449, 11, 1186, 1640, 134, 6, 158, 3256, 50118, 50117, 50117, 1594, 1640, 118, 3226, 330, 45994, 234, 3256, 50118, 50117, 50117, 50117, 17265, 45803, 9904, 27645, 50118, 50117, 50117, 50117, 44054, 1640, 288, 43, 50118, 17265, 45803, 3084, 27645, 50118, 2, 2, 41975, 47427, 50118, 46797, 5457, 47427, 4, 47717, 179, 4, 47438, 4, 12745, 1902, 50118, 500, 6, 347, 6, 530, 5457, 5456, 1640, 2544, 6, 46797, 49123, 44154, 49338, 50118, 8396, 29, 5457, 48395, 288, 742, 3226, 1640, 347, 2744, 134, 43, 13, 18134, 11, 1186, 1640, 500, 2744, 134, 46077, 50118, 2444, 597, 5457, 158, 12606, 1366, 50118, 1990, 939, 11, 1186, 1640, 530, 3256, 50118, 50117, 1069, 6, 2520, 6, 6873, 5457, 5456, 1640, 2544, 6, 46797, 49123, 44154, 49338, 50118, 50117, 8396, 29, 10975, 1069, 46386, 2520, 742, 5457, 12987, 50118, 5174, 5457, 48395, 10975, 12, 2444, 597, 742, 3226, 1640, 

In [9]:
encoded_dataset = encoded_dataset.train_test_split(0.1,seed=100)

In [10]:
from transformers import AdamW
import transformers
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8, 
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred): #이게 필요한가?
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [13]:
'''
class TrainerModified(Trainer):
  def compute_loss(self,model,inputs,return_outputs=False):
    labels = inputs.get("labels")
    outputs = model()
'''

'\nclass TrainerModified(Trainer):\n  def compute_loss(self,model,inputs,return_outputs=False):\n    labels = inputs.get("labels")\n    outputs = model()\n'

In [14]:
args = TrainingArguments("test", save_strategy="epoch",evaluation_strategy="epoch",logging_strategy="epoch", 
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model="accuracy",load_best_model_at_end=True)

trainer = Trainer(model,args,train_dataset=encoded_dataset['train'],eval_dataset=encoded_dataset['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)


In [15]:
gc.collect()
torch.cuda.empty_cache()

In [16]:
trainer.train()

***** Running training *****
  Num examples = 40579
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 17759


Epoch,Training Loss,Validation Loss,Accuracy
1,0.269900,0.145242,0.952096
2,0.146900,0.158933,0.956310
3,0.110700,0.203906,0.956088
4,0.083300,0.218258,0.956531
5,0.060300,0.188926,0.969173
6,0.046500,0.172191,0.970725
7,0.040300,0.194919,0.969173


***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16
Saving model checkpoint to test/checkpoint-2537
Configuration saved in test/checkpoint-2537/config.json
Model weights saved in test/checkpoint-2537/pytorch_model.bin
tokenizer config file saved in test/checkpoint-2537/tokenizer_config.json
Special tokens file saved in test/checkpoint-2537/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16
Saving model checkpoint to test/checkpoint-5074
Configuration saved in test/checkpoint-5074/config.json
Model weights saved in test/checkpoint-5074/pytorch_model.bin
tokenizer config file saved in test/checkpoint-5074/tokenizer_config.json
Special tokens file saved in test/checkpoint-5074/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16
Saving model checkpoint to test/checkpoint-7611
Configuration saved in test/checkpoint-7611/config.json
Model weights saved in test/checkpoint-7611/pytorch_mod

TrainOutput(global_step=17759, training_loss=0.10828670611473627, metrics={'train_runtime': 15893.4995, 'train_samples_per_second': 17.872, 'train_steps_per_second': 1.117, 'total_flos': 7.473748460817408e+16, 'train_loss': 0.10828670611473627, 'epoch': 7.0})

In [17]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16


{'epoch': 7.0,
 'eval_accuracy': 0.9707252162341983,
 'eval_loss': 0.1721908301115036,
 'eval_runtime': 80.8684,
 'eval_samples_per_second': 55.757,
 'eval_steps_per_second': 3.487}

In [18]:
#test data preprocess 추가
TEST = "test.csv"
SAMPLE = "sample_submission.csv"

test = pd.read_csv('test.csv', encoding = 'utf-8')
df_test = pd.DataFrame(test)

preprocess(df_test,"preprocess_test.csv")

test_dataset = load_dataset("csv",data_files = "preprocess_test.csv")['train']
test_dataset = test_dataset.map(tokenized,remove_columns=['code1','code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SAMPLE)
df['similar'] = np.argmax(predictions.predictions,axis=-1)
df.to_csv('/content/drive/MyDrive/submission.csv',index=False)

Using custom data configuration default-9616f41dd76fddf6


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9616f41dd76fddf6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/179700 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0, pair_id. If Unnamed: 0, pair_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 179700
  Batch size = 16
